# Setup

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#required libraries to get an overview of dataset with Python

In [79]:
def extract(filename):
    file = "/files/Macroeconometrics/Temperature/dat/" + filename  # identify location
    text = open(file, 'r')  #open text file
    
    table=[]  #create an empty table where relevant data will be inserted
    for line in text:
        column = line.split()   #split text file into words using the in-built split function
        if line[0] == '#':   #ignore comments
            continue
        elif len(column) == 0: #ignore empty lines
            continue
        elif line[0] == 'S':  #Ignore first line of string (description of dataset. Line starts with the word "Switzerland")
            continue
        elif line[2] == 'Y': #Ignore column names (First column is "Year")
            continue
        else:
            year = int(column[0])  #define columns and name them
            age = column[1]  
            female = column[2]
            male = column[3]
            total = column[4]
            table.append([year, age, female, male, total])  #insert variables into table
    text.close()
    return(table)

# Mortality (absoulte)

In [80]:
x = extract("Deaths_5x1.txt")   #Turns .txt file into a list that we can then turn into pandas dataframe

In [81]:
death = pd.DataFrame(x, columns=['Year', 'Age', 'Female', 'Male', 'Total'])

In [82]:
death = death.rename(columns={"Female": "Fem_death", "Male": "Male_death", "Total":"Total_death"})

In [83]:
death

,Year,Age,Fem_death,Male_death,Total_death
0,1876,0,7982.00,10059.00,18041.00
1,1876,1-4,2566.00,2562.00,5128.00
2,1876,5-9,787.00,839.00,1626.00
3,1876,10-14,560.00,477.00,1037.00
4,1876,15-19,754.00,731.00,1485.00
...,...,...,...,...,...
3523,2022,90-94,8305.00,4836.00,13141.00
3524,2022,95-99,4148.00,1602.00,5750.00
3525,2022,100-104,764.00,199.00,963.00
3526,2022,105-109,50.00,11.00,61.00


# Population

In [84]:
y = extract("Population5.txt")

In [85]:
pop = pd.DataFrame(y, columns=['Year', 'Age', 'Female', 'Male', 'Total'])

In [86]:
pop.drop(pop.tail(24).index,inplace=True)    #Data goes up to 2023 while other datasets go up to 2022, so we drop the last year from the data

In [87]:
pop = pop.drop(["Age","Year"], axis=1)       #These column are already present in the absolute death dataframe. Since we will join these two tables together, we don't need these columns

In [88]:
pop = pop.rename(columns={"Female": "Fem_pop", "Male": "Male_pop", "Total":"Total_pop"})      #Renaming columns

In [89]:
pop

,Fem_pop,Male_pop,Total_pop
0,33781.00,34056.00,67837.00
1,129479.00,131343.00,260822.00
2,143808.00,141747.00,285555.00
3,137624.00,135164.00,272788.00
4,125369.00,121460.00,246829.00
...,...,...,...
3523,46441.25,21421.94,67863.19
3524,13034.00,4232.88,17266.88
3525,1544.50,357.02,1901.52
3526,71.73,14.50,86.23


# Mortality Rate

*You will notice some columns will start with capital letters and others in lower case. I did this to differentiate between string values and float values. In the original dataset, most values were stored as strings since we imported a .txt file. For these values I named the columns with capital letters. However, after converting the datatype to float values I also turned the columns into lower case to differentiate the two.*

In [90]:
df_merged = pd.concat([death, pop], ignore_index=False, sort=False, axis=1)

In [91]:
df_merged

,Year,Age,Fem_death,Male_death,Total_death,Fem_pop,Male_pop,Total_pop
0,1876,0,7982.00,10059.00,18041.00,33781.00,34056.00,67837.00
1,1876,1-4,2566.00,2562.00,5128.00,129479.00,131343.00,260822.00
2,1876,5-9,787.00,839.00,1626.00,143808.00,141747.00,285555.00
3,1876,10-14,560.00,477.00,1037.00,137624.00,135164.00,272788.00
4,1876,15-19,754.00,731.00,1485.00,125369.00,121460.00,246829.00
...,...,...,...,...,...,...,...,...
3523,2022,90-94,8305.00,4836.00,13141.00,46441.25,21421.94,67863.19
3524,2022,95-99,4148.00,1602.00,5750.00,13034.00,4232.88,17266.88
3525,2022,100-104,764.00,199.00,963.00,1544.50,357.02,1901.52
3526,2022,105-109,50.00,11.00,61.00,71.73,14.50,86.23


In [92]:
df_merged['total_pop'] = df_merged['Total_pop'].astype(float)
df_merged['fem_pop'] = df_merged['Fem_pop'].astype(float)
df_merged['male_pop'] = df_merged['Male_pop'].astype(float)
df_merged['total_death'] = df_merged['Total_death'].astype(float)
df_merged['fem_death'] = df_merged['Fem_death'].astype(float)
df_merged['male_death'] = df_merged['Male_death'].astype(float)

In [93]:
df_merged['Grouped Age'] = 'Other'

df_merged.loc[df_merged['Age'] == '0', 'Grouped Age'] = '0-24'
df_merged.loc[df_merged['Age'] == '1-4', 'Grouped Age'] = '0-24'
df_merged.loc[df_merged['Age'] == '5-9', 'Grouped Age'] = '0-24'                   #This block of code creates a new 'Grouped Age' column and groups the age ranges given by the dataset into 3 age groups: 0-24, 25-65 and 65+
df_merged.loc[df_merged['Age'] == '10-14', 'Grouped Age'] = '0-24'
df_merged.loc[df_merged['Age'] == '15-19', 'Grouped Age'] = '0-24'
df_merged.loc[df_merged['Age'] == '20-24', 'Grouped Age'] = '0-24'

df_merged.loc[df_merged['Age'] == '25-29', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '30-34', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '35-39', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '40-44', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '45-49', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '50-54', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '55-59', 'Grouped Age'] = '24-64'
df_merged.loc[df_merged['Age'] == '60-64', 'Grouped Age'] = '24-64'

df_merged.loc[df_merged['Age'] == '65-69', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '70-74', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '75-79', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '80-84', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '85-89', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '90-94', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '95-99', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '100-104', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '105-109', 'Grouped Age'] = '65+'
df_merged.loc[df_merged['Age'] == '110+', 'Grouped Age'] = '65+'

In [94]:
df_merged = df_merged.drop(['Fem_death','Male_death','Total_death','Fem_pop','Male_pop','Total_pop'], axis=1)    #dropping columns with string datatype

In [95]:
df_merged.head(24)

,Year,Age,total_pop,fem_pop,male_pop,total_death,fem_death,male_death,Grouped Age
0,1876,0,67837.00,33781.00,34056.00,18041.0,7982.0,10059.0,0-24
1,1876,1-4,260822.00,129479.00,131343.00,5128.0,2566.0,2562.0,0-24
2,1876,5-9,285555.00,143808.00,141747.00,1626.0,787.0,839.0,0-24
3,1876,10-14,272788.00,137624.00,135164.00,1037.0,560.0,477.0,0-24
4,1876,15-19,246829.00,125369.00,121460.00,1485.0,754.0,731.0,0-24
5,1876,20-24,216100.00,110514.00,105586.00,1698.0,831.0,867.0,0-24
6,1876,25-29,205146.00,106476.00,98670.00,1899.0,943.0,956.0,24-64
7,1876,30-34,199292.00,103386.00,95906.00,2099.0,1044.0,1055.0,24-64
8,1876,35-39,186646.00,96655.00,89991.00,2349.0,1159.0,1190.0,24-64
9,1876,40-44,167366.00,86185.00,81181.00,2269.0,1010.0,1259.0,24-64


In [96]:
p1 = df_merged.groupby(['Year','Grouped Age'])['total_pop'].sum().reset_index()      #Here, we create new dataframes where the different metrics are summed for each different age group across each year

p2 = df_merged.groupby(['Year','Grouped Age'])['fem_pop'].sum().reset_index()

p3 = df_merged.groupby(['Year','Grouped Age'])['male_pop'].sum().reset_index()

d1 = df_merged.groupby(['Year','Grouped Age'])['total_death'].sum().reset_index()

d2 = df_merged.groupby(['Year','Grouped Age'])['fem_death'].sum().reset_index()

d3 = df_merged.groupby(['Year','Grouped Age'])['male_death'].sum().reset_index()

In [97]:
p2 = p2.drop(['Grouped Age', 'Year'], axis=1)        #Dropping shared columns
p3 = p3.drop(['Grouped Age', 'Year'], axis=1)
d1 = d1.drop(['Grouped Age', 'Year'], axis=1)
d2 = d2.drop(['Grouped Age', 'Year'], axis=1)
d3 = d3.drop(['Grouped Age', 'Year'], axis=1)


In [98]:
df_merged_v2 = pd.concat([p1, p2, p3, d1, d2, d3], ignore_index=False, sort=False, axis=1)        #Combine the new dataframes for grouped ages

In [99]:
df_merged_v2.head(20)

,Year,Grouped Age,total_pop,fem_pop,male_pop,total_death,fem_death,male_death
0,1876,0-24,1349931.00,680575.00,669356.00,29015.00,13480.0,15535.00
1,1876,24-64,1248984.00,647407.00,601577.00,21958.00,10280.0,11678.00
2,1876,65+,149006.49,76654.00,72352.49,15846.00,7939.0,7907.00
3,1877,0-24,1363749.00,687192.00,676557.00,30429.00,14187.0,16242.00
4,1877,24-64,1252775.00,649420.00,603355.00,20147.00,9693.0,10454.00
5,1877,65+,149627.44,77169.43,72458.01,14777.00,7547.0,7230.00
6,1878,0-24,1376956.00,693685.00,683271.00,29486.00,13953.0,15533.00
7,1878,24-64,1256613.00,651316.00,605297.00,20203.00,9679.0,10524.00
8,1878,65+,150903.40,78007.07,72896.33,15622.00,8048.0,7574.00
9,1879,0-24,1389040.00,700182.00,688858.00,26540.00,12392.0,14148.00


In [100]:
df_merged_v2['total_mortality'] = round(df_merged_v2['total_death']/df_merged_v2['total_pop'],5)       #creating columns for mortality rate. I round it to 5 decimal places
df_merged_v2['fem_mortality'] = round(df_merged_v2['fem_death']/df_merged_v2['fem_pop'],5)
df_merged_v2['male_mortality'] = round(df_merged_v2['male_death']/df_merged_v2['male_pop'],5)

In [101]:
df_merged_v2.head(20)

,Year,Grouped Age,total_pop,fem_pop,male_pop,total_death,fem_death,male_death,total_mortality,fem_mortality,male_mortality
0,1876,0-24,1349931.00,680575.00,669356.00,29015.00,13480.0,15535.00,0.02149,0.01981,0.02321
1,1876,24-64,1248984.00,647407.00,601577.00,21958.00,10280.0,11678.00,0.01758,0.01588,0.01941
2,1876,65+,149006.49,76654.00,72352.49,15846.00,7939.0,7907.00,0.10634,0.10357,0.10928
3,1877,0-24,1363749.00,687192.00,676557.00,30429.00,14187.0,16242.00,0.02231,0.02064,0.02401
4,1877,24-64,1252775.00,649420.00,603355.00,20147.00,9693.0,10454.00,0.01608,0.01493,0.01733
5,1877,65+,149627.44,77169.43,72458.01,14777.00,7547.0,7230.00,0.09876,0.09780,0.09978
6,1878,0-24,1376956.00,693685.00,683271.00,29486.00,13953.0,15533.00,0.02141,0.02011,0.02273
7,1878,24-64,1256613.00,651316.00,605297.00,20203.00,9679.0,10524.00,0.01608,0.01486,0.01739
8,1878,65+,150903.40,78007.07,72896.33,15622.00,8048.0,7574.00,0.10352,0.10317,0.10390
9,1879,0-24,1389040.00,700182.00,688858.00,26540.00,12392.0,14148.00,0.01911,0.01770,0.02054


In [102]:
df_merged_v2 = pd.get_dummies(df_merged_v2, columns = ['Grouped Age'], drop_first = False)         #creates dummy variables for age groups that we can use in regression. Also drops the original age column as it is no longer necessary.

In [103]:
df_merged_v2

,Year,total_pop,fem_pop,male_pop,total_death,fem_death,male_death,total_mortality,fem_mortality,male_mortality,Grouped Age_0-24,Grouped Age_24-64,Grouped Age_65+
0,1876,1349931.00,680575.00,669356.00,29015.0,13480.0,15535.0,0.02149,0.01981,0.02321,1,0,0
1,1876,1248984.00,647407.00,601577.00,21958.0,10280.0,11678.0,0.01758,0.01588,0.01941,0,1,0
2,1876,149006.49,76654.00,72352.49,15846.0,7939.0,7907.0,0.10634,0.10357,0.10928,0,0,1
3,1877,1363749.00,687192.00,676557.00,30429.0,14187.0,16242.0,0.02231,0.02064,0.02401,1,0,0
4,1877,1252775.00,649420.00,603355.00,20147.0,9693.0,10454.0,0.01608,0.01493,0.01733,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,2021,4836148.00,2398506.00,2437642.00,8632.0,3121.0,5511.0,0.00178,0.00130,0.00226,0,1,0
437,2021,1629456.00,899071.04,730384.96,61933.0,32729.0,29204.0,0.03801,0.03640,0.03998,0,0,1
438,2022,2214500.00,1074832.00,1139668.00,658.0,260.0,398.0,0.00030,0.00024,0.00035,1,0,0
439,2022,4862972.00,2411583.00,2451389.00,8516.0,3099.0,5417.0,0.00175,0.00129,0.00221,0,1,0


In [104]:
df_merged_v2.to_csv('Mortality_v2.csv', sep=',', index=False, encoding='utf-8')